In [34]:
import numpy as np

class Convolution:
    #include init? idk
        
    # Given set of filters (features), iterate over input and multiply
    # img_in: a NxM matrix representing one of the RGB values of the input
    # features: filters/features that we compare the input against
    # returns a PxQ matrix (P<N, Q<M) where each value is the result of multiplying
    # filter with the input, then dividing by the size of the filter
    def forward_prop(self, img_in, features, step_size):
        N = len(img_in)
        M = len(img_in[0])
        # if step size doesn't match up -> padding()
        
        # gives us amount of padding needed
        pad_N = (N - len(features)) % step_size
        pad_M = (M - len(features[0])) % step_size
        
        img_in = self.padding(img_in, N+pad_N, M+pad_M) #padded img
        
        # should be whole numbers
        P = (len(img_in) - len(features)) / step_size + 1
        Q = (len(img_in[0]) - len(features)) / step_size + 1
        
        # find good ways to add padding, possibly this part is not needed?
        if P.is_integer() is False:
            return "Bad Dimension P"
        if Q.is_integer() is False:
            return "Bad dimension Q"
        else:
            P = int(P)
            Q = int(Q)
            
        print(P, Q)
        out = np.zeros((P, Q)) # PxQ output array
        
        # iterate thru the input and output to corresponding part in out
        for i in range(0, P):
            for j in range (0, Q):
                # Take dot product of the flattened filter and part it is covering
                out[i, j] = np.dot(np.ndarray.flatten(img_in[i:i+len(features),
                            j:j+len(features[0])]), np.ndarray.flatten(features))
        
        return out
    def back_prop(): 
        return None
    
    # Add 0s to the N and M sides of the input as needed
    def padding(self, orig_img, N, M):
        pad_arr = np.zeros((N, M))
        pad_arr[:orig_img.shape[0], :orig_img.shape[1]] = orig_img
        return pad_arr
    
class Pooling: 
    def forward_prop():
        return None
    def back_prop():
        return None
    
class ReLU_Act():
    def forward_prop():
        return None
    def back_prop():
        return None
    
class Sigmoid_Act():
    def forward_prop():
        return None
    def back_prop():
        return None
    
class Flatten_Matrix(): 
    def forward_prop():
        return None
    def back_prop():
        return None
          

In [42]:
# tests
def test_conv_forward():
    test_arr = np.array(([5, 5, 2],
                         [5, 5, 5],
                         [5, 5, 5]))
    feat_arr = np.array(([2, 2], 
                         [2, 2]))
    step = 2
    conv_layer = Convolution()
    print(conv_layer.forward_prop(test_arr, feat_arr, step))
    
test_conv_forward()

2 2
[[ 40.  34.]
 [ 40.  40.]]
